# Creare nuove feature di storicità e attualità

Crea sempre nuove feature, poi fomatta e fai gli swap
Mantenere la storicità è più facili con le supposizioni

In [1]:
# LIBRERIE
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from pandas.api.types import CategoricalDtype
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [2]:
# CARICAMENTO DATI
def loadDataset(year):
    dataset_path = "../datasets/tennis/"+year+".xlsx"
    return pd.read_excel(dataset_path)

In [35]:

def generalFormatting(dfCopy, playerEncode = 0):
    
    df = dfCopy.copy()
    
    # Cambio nome delle colonne Winner/Loser
    dictCol = {'Winner' :"Player1", 'Loser':'Player2', 'WRank':'P1_Rank', 'LRank':'P2_Rank', 'WPts':'P1_Pts', 'LPts':'P2_Pts',
       'W1':'P1_S1', 'L1':'P2_S1', 'W2':'P1_S2', "L2":'P2_S2', 'W3':'P1_S3', 'L3':'P2_S3', 'W4':'P1_S4', 'L4':'P2_S4', 'W5':'P1_S5', "L5":'P2_S5', 'Wsets':'P1_sets',
       'Lsets':'P2_sets', 'B365W':'P1_B365', 'B365L':'P2_B365', 'EXW':'P1_EX', 'EXL':'P2_EX', 'LBW':'P1_LB', 'LBL':'P2_LB', 'PSW':'P1_PS',
       'PSL':'P2_PS', 'MaxW':'P1_Max', 'MaxL':'P2_Max', 'AvgW':'P1_Avg', 'AvgL':'P2_Avg'}

    df.rename(columns=dictCol, inplace=True)
    
    # Fill NA
    df.fillna(0, inplace = True)
    
    # One Hot Encoding
    df = pd.get_dummies(df, columns = ["Court", "Surface","Comment", "Best of"])
    
    # Encoding Tipo torneo con peso
    df["Series"]=df["Series"].astype('category')
    cats_to_order = ["ATP250", "ATP500", "Masters 1000", "Masters Cup", "Grand Slam"]
    covered_type = CategoricalDtype(categories=cats_to_order, ordered=True)
    df['Series'] = df['Series'].cat.reorder_categories(cats_to_order, ordered=True)
    df["Series"] = df["Series"].cat.codes
    
    # Encoding Round Torneo con peso
    df["Round"]=df["Round"].astype('category')
    cats_to_order = ["Round Robin", "1st Round",'2nd Round','3rd Round','4th Round','Quarterfinals', 'Semifinals', 'The Final']
    covered_type = CategoricalDtype(categories=cats_to_order, ordered=True)
    df['Round'] = df['Round'].cat.reorder_categories(cats_to_order, ordered=True)
    df["Round"] = df["Round"].cat.codes
    
    # Rimozione colonne influenti
    df.drop(["Location", "ATP"], axis =1, inplace=True)
    
    # Encoding Nome tornei
    lb_make = LabelEncoder()
    df["Tournament"] = lb_make.fit_transform(df["Tournament"])
    df["target"] = 0
    
    return df
    


In [36]:
prec = loadDataset("2016")
now = loadDataset("2017")

In [37]:
prec = generalFormatting(prec)
now = generalFormatting(now)

# Vittorie in quel tipo di campo

In [6]:
# Non è propriamente corretto
def fieldVictory(old, new):
    playerList=set(new["Player1"]) | set(new["Player2"])
    print(len(playerList))
    subDf = pd.DataFrame(data = {"player":list(playerList), "hardWin": 1, "clayWin":1, "grassWin":1})
    for index, row in old.iterrows():
        if row["Player1"] in playerList:
            subDf.loc[subDf["player"]==row["Player1"],"hardWin"]+=row["Surface_Hard"]
            subDf.loc[subDf["player"]==row["Player1"],"clayWin"]+=row["Surface_Clay"]
            subDf.loc[subDf["player"]==row["Player1"],"grassWin"]+=row["Surface_Grass"]
        else:
            subDf.loc[subDf["player"]==row["Player1"],"hardWin"]+=1
            subDf.loc[subDf["player"]==row["Player1"],"clayWin"]+=1
            subDf.loc[subDf["player"]==row["Player1"],"grassWin"]+=1
        
    for index, row in subDf.iterrows():
        frac  = row["hardWin"]+row["clayWin"]+row["grassWin"]
        subDf.loc[index,"hardWin"] = row["hardWin"]/frac
        subDf.loc[index,"clayWin"] = row["clayWin"]/frac
        subDf.loc[index,"grassWin"] = row["grassWin"]/frac
        
    return subDf

In [7]:
aaa = fieldVictory(prec, now)
aaa

307


,player,hardWin,clayWin,grassWin
0,Clezar G.,0.333333,0.333333,0.333333
1,Daniel T.,0.222222,0.666667,0.111111
2,Bourgue M.,0.250000,0.500000,0.250000
3,Wu Y.,0.333333,0.333333,0.333333
4,Tearney F.,0.333333,0.333333,0.333333
...,...,...,...,...
302,Sakharov G.,0.333333,0.333333,0.333333
303,Garcia-Lopez G.,0.400000,0.520000,0.080000
304,Menendez-Maceiras A.,0.333333,0.333333,0.333333
305,Ilhan M.,0.333333,0.333333,0.333333


In [8]:

def addFieldWin(now, fwlr):
    now["P1_winningField"] = 0
    now["P2_winningField"] = 0

    for index, row in now.iterrows():
        if now.loc[index, "Surface_Hard"] == 1:
            value1=fwlr[fwlr["player"] == row["Player1"]]["hardWin"].tolist()[0]
            now.loc[index,"P1_winningField"] = value1

            value2=fwlr[fwlr["player"] == row["Player2"]]["hardWin"].tolist()[0]
            now.loc[index,"P2_winningField"] = value2

        elif now.loc[index, "Surface_Grass"] == 1:
            value=fwlr[fwlr["player"] == row["Player1"]]["grassWin"].tolist()[0]
            now.loc[index,"P1_winningField"] = value1

            value2=fwlr[fwlr["player"] == row["Player2"]]["grassWin"].tolist()[0]
            now.loc[index,"P2_winningField"] = value2

        elif now.loc[index, "Surface_Clay"] == 1:
            value=fwlr[fwlr["player"] == row["Player1"]]["clayWin"].tolist()[0]
            now.loc[index,"P1_winningField"] = value1

            value2=fwlr[fwlr["player"] == row["Player2"]]["clayWin"].tolist()[0]
            now.loc[index,"P2_winningField"] = value2
        


In [9]:
addFieldWin(prec, now)

307


In [10]:
asd = aaa[aaa["player"] == row["Player1"]]["clayWin"]
asd

NameError: name 'row' is not defined

In [11]:
now

,Tournament,Date,Series,Round,Player1,Player2,P1_Rank,P2_Rank,P1_Pts,P2_Pts,...,Surface_Grass,Surface_Hard,Comment_Completed,Comment_Retired,Comment_Walkover,Best of_3,Best of_5,target,P1_winningField,P2_winningField
0,15,2017-01-01,0,1,Thompson J.,Ymer E.,79,160.0,689,372.0,...,0,1,1,0,0,1,0,0,0.500000,0.200000
1,15,2017-01-02,0,1,Mahut N.,Robert S.,39,54.0,1102,854.0,...,0,1,1,0,0,1,0,0,0.538462,0.470588
2,15,2017-01-02,0,1,Ferrer D.,Tomic B.,21,26.0,1785,1465.0,...,0,1,1,0,0,1,0,0,0.472222,0.645161
3,15,2017-01-02,0,1,Edmund K.,Escobedo E.,45,141.0,1001,443.0,...,0,1,1,0,0,1,0,0,0.636364,0.400000
4,15,2017-01-02,0,1,Dimitrov G.,Johnson S.,17,33.0,2035,1320.0,...,0,1,1,0,0,1,0,0,0.813953,0.533333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628,39,2017-11-17,3,0,Goffin D.,Thiem D.,8,4.0,2975,3815.0,...,0,1,1,0,0,1,0,0,0.638298,0.426230
2629,39,2017-11-17,3,0,Dimitrov G.,Carreno Busta P.,6,10.0,3650,2615.0,...,0,1,1,0,0,1,0,0,0.813953,0.555556
2630,39,2017-11-18,3,6,Goffin D.,Federer R.,8,2.0,2975,9005.0,...,0,1,1,0,0,1,0,0,0.638298,0.375000
2631,39,2017-11-18,3,6,Dimitrov G.,Sock J.,6,9.0,3650,2765.0,...,0,1,1,0,0,1,0,0,0.813953,0.692308


# Partite vinte prima dell'incontro

In [12]:
from function import randomSwap

In [13]:
randomSwap(now)
now

,Tournament,Date,Series,Round,Player1,Player2,P1_Rank,P2_Rank,P1_Pts,P2_Pts,...,Surface_Grass,Surface_Hard,Comment_Completed,Comment_Retired,Comment_Walkover,Best of_3,Best of_5,target,P1_winningField,P2_winningField
0,15,2017-01-01,0,1,Thompson J.,Ymer E.,79,160.0,689,372.0,...,0,1,1,0,0,1,0,0,0.500000,0.200000
1,15,2017-01-02,0,1,Mahut N.,Robert S.,39,54.0,1102,854.0,...,0,1,1,0,0,1,0,0,0.538462,0.470588
2,15,2017-01-02,0,1,Ferrer D.,Tomic B.,21,26.0,1785,1465.0,...,0,1,1,0,0,1,0,1,0.472222,0.645161
3,15,2017-01-02,0,1,Edmund K.,Escobedo E.,45,141.0,1001,443.0,...,0,1,1,0,0,1,0,1,0.636364,0.400000
4,15,2017-01-02,0,1,Dimitrov G.,Johnson S.,17,33.0,2035,1320.0,...,0,1,1,0,0,1,0,0,0.813953,0.533333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628,39,2017-11-17,3,0,Goffin D.,Thiem D.,8,4.0,2975,3815.0,...,0,1,1,0,0,1,0,0,0.638298,0.426230
2629,39,2017-11-17,3,0,Dimitrov G.,Carreno Busta P.,6,10.0,3650,2615.0,...,0,1,1,0,0,1,0,0,0.813953,0.555556
2630,39,2017-11-18,3,6,Goffin D.,Federer R.,8,2.0,2975,9005.0,...,0,1,1,0,0,1,0,1,0.638298,0.375000
2631,39,2017-11-18,3,6,Dimitrov G.,Sock J.,6,9.0,3650,2765.0,...,0,1,1,0,0,1,0,0,0.813953,0.692308


In [14]:
# Post swap 

# Seems work

def precWin(new):
    playerList=set(new["Player1"]) | set(new["Player2"])
    sub = pd.DataFrame(data = {"player":list(playerList), "P1_precWin":0, "P2_precWin":0, "P1_precLose":0, "P2_precLose":0})
    
    new["P1_precWin"], new["P2_precWin"], new["P1_precLose"], new["P2_precLose"] = [0,0,0,0]
    for index, row in new.iterrows():
        new.loc[index,"P1_precWin"] = sub[sub["player"] == row["Player1"]]["P1_precWin"].tolist()[0]
        new.loc[index,"P1_precLose"] = sub[sub["player"] == row["Player1"]]["P1_precLose"].tolist()[0]
        new.loc[index,"P2_precLose"] = sub[sub["player"] == row["Player2"]]["P2_precLose"].tolist()[0]
        new.loc[index,"P2_precWin"] = sub[sub["player"] == row["Player2"]]["P2_precWin"].tolist()[0]
               
        if row["target"] == 0:
            sub.loc[sub["player"]==row["Player1"],"P1_precWin"]+=1
            sub.loc[sub["player"]==row["Player1"],"P1_precLose"] = 0    
            sub.loc[sub["player"]==row["Player2"],"P2_precLose"]+=1
            sub.loc[sub["player"]==row["Player2"],"P2_precWin"] = 0
            
        else:
            sub.loc[sub["player"] == row["Player2"],"P2_precWin"] += 1
            sub.loc[sub["player"]==row["Player2"],"P2_precLose"] = 0
            sub.loc[sub["player"] == row["Player1"],"P1_precLose"] += 1
            sub.loc[sub["player"]==row["Player1"],"P1_precWin"] = 0
        
    

In [15]:
precWin(now)

In [16]:
now

,Tournament,Date,Series,Round,Player1,Player2,P1_Rank,P2_Rank,P1_Pts,P2_Pts,...,Comment_Walkover,Best of_3,Best of_5,target,P1_winningField,P2_winningField,P1_precWin,P2_precWin,P1_precLose,P2_precLose
0,15,2017-01-01,0,1,Thompson J.,Ymer E.,79,160.0,689,372.0,...,0,1,0,0,0.500000,0.200000,0,0,0,0
1,15,2017-01-02,0,1,Mahut N.,Robert S.,39,54.0,1102,854.0,...,0,1,0,0,0.538462,0.470588,0,0,0,0
2,15,2017-01-02,0,1,Ferrer D.,Tomic B.,21,26.0,1785,1465.0,...,0,1,0,1,0.472222,0.645161,0,0,0,0
3,15,2017-01-02,0,1,Edmund K.,Escobedo E.,45,141.0,1001,443.0,...,0,1,0,1,0.636364,0.400000,0,0,0,0
4,15,2017-01-02,0,1,Dimitrov G.,Johnson S.,17,33.0,2035,1320.0,...,0,1,0,0,0.813953,0.533333,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628,39,2017-11-17,3,0,Goffin D.,Thiem D.,8,4.0,2975,3815.0,...,0,1,0,0,0.638298,0.426230,2,1,0,0
2629,39,2017-11-17,3,0,Dimitrov G.,Carreno Busta P.,6,10.0,3650,2615.0,...,0,1,0,0,0.813953,0.555556,1,2,0,0
2630,39,2017-11-18,3,6,Goffin D.,Federer R.,8,2.0,2975,9005.0,...,0,1,0,1,0.638298,0.375000,3,2,0,0
2631,39,2017-11-18,3,6,Dimitrov G.,Sock J.,6,9.0,3650,2765.0,...,0,1,0,0,0.813953,0.692308,2,0,0,1


In [17]:
now.loc[:,["Date","Tournament", "Round","Player1","Player2" ,"target"]]

,Date,Tournament,Round,Player1,Player2,target
0,2017-01-01,15,1,Thompson J.,Ymer E.,0
1,2017-01-02,15,1,Mahut N.,Robert S.,0
2,2017-01-02,15,1,Ferrer D.,Tomic B.,1
3,2017-01-02,15,1,Edmund K.,Escobedo E.,1
4,2017-01-02,15,1,Dimitrov G.,Johnson S.,0
...,...,...,...,...,...,...
2628,2017-11-17,39,0,Goffin D.,Thiem D.,0
2629,2017-11-17,39,0,Dimitrov G.,Carreno Busta P.,0
2630,2017-11-18,39,6,Goffin D.,Federer R.,1
2631,2017-11-18,39,6,Dimitrov G.,Sock J.,0


In [ ]:
now[now["P2_precWin"] > 3]

In [ ]:
now["target"].value_counts()

In [18]:
aaa = loadDataset("2017")

In [23]:
aaa.loc[:,["Winner","Loser","WRank","LRank"]]

,Winner,Loser,WRank,LRank
0,Thompson J.,Ymer E.,79,160.0
1,Mahut N.,Robert S.,39,54.0
2,Ferrer D.,Tomic B.,21,26.0
3,Edmund K.,Escobedo E.,45,141.0
4,Dimitrov G.,Johnson S.,17,33.0
...,...,...,...,...
2628,Goffin D.,Thiem D.,8,4.0
2629,Dimitrov G.,Carreno Busta P.,6,10.0
2630,Goffin D.,Federer R.,8,2.0
2631,Dimitrov G.,Sock J.,6,9.0


# Field Win all years

In [128]:
now = loadDataset("2019")
now["Surface"].value_counts()
# now = generalFormatting(now)

Hard     1503
Clay      783
Grass     324
Name: Surface, dtype: int64

In [127]:
def fieldWinLose ():
    allCsv = pd.DataFrame(columns = ["Winner","Loser", "Surface"])
    
    for i in range(1,9):
        df = pd.read_excel("../datasets/tennis/200"+str(i)+".xls")
        df= df.loc[:,["Winner", "Loser","Surface"]]        
        frames = [allCsv, df]
        allCsv = pd.concat(frames, sort = False)
        
    for i in range(0,8):
        df = pd.read_excel("../datasets/tennis/201"+str(i)+".xlsx")
        df= df.loc[:,["Winner", "Loser","Surface"]]        
        frames = [allCsv, df]
        allCsv = pd.concat(frames, sort = False)
        
    
    return allCsv

def fieldWinLoseRateo(df, now):
    playerList=set(df["Winner"]) | set(df["Loser"]) | set(now["Winner"]) | set(now["Loser"])
    print(len(playerList))
    subDf = pd.DataFrame(data = {"player":list(playerList), "hardWin": 1, "clayWin":1, "grassWin":1, "hardLose":1,
                                                             "clayLose":1, "grassLose":1})
    for index, row in df.iterrows():
        if row["Winner"] in playerList:
            subDf.loc[subDf["player"]==row["Winner"],"hardWin"]+=row["Surface"] == "Hard"
            subDf.loc[subDf["player"]==row["Winner"],"clayWin"]+=row["Surface"] == "Clay"
            subDf.loc[subDf["player"]==row["Winner"],"grassWin"]+=row["Surface"] == "Grass"
        elif row["Loser"] in playerList:
            subDf.loc[subDf["player"]==row["Loser"],"hardLose"]+=row["Surface"] == "Hard"
            subDf.loc[subDf["player"]==row["Loser"],"clayLose"]+=row["Surface"] == "Clay"
            subDf.loc[subDf["player"]==row["Loser"],"grassLose"]+=row["Surface"] == "Grass"
        else:    
            subDf.loc[subDf["player"]==row["Winner"],"hardWin"]+=1
            subDf.loc[subDf["player"]==row["Winner"],"clayWin"]+=1
            subDf.loc[subDf["player"]==row["Winner"],"grassWin"]+=1
            subDf.loc[subDf["player"]==row["Loser"],"hardLose"]+=1
            subDf.loc[subDf["player"]==row["Loser"],"clayLose"]+=1
            subDf.loc[subDf["player"]==row["Loser"],"grassLose"]+=1
            
        
    for index, row in subDf.iterrows():
        subDf.loc[index,"hardWin"] = row["hardWin"]/(row["hardWin"] + row["hardLose"])
        subDf.loc[index,"clayWin"] = row["clayWin"]/(row["clayWin"] + row["clayLose"])
        subDf.loc[index,"grassWin"] = row["grassWin"]/(row["grassWin"] + row["grassLose"])
        
        subDf.loc[index,"hardLose"] = row["hardLose"]/(row["hardWin"] + row["hardLose"])
        subDf.loc[index,"clayLose"] = row["clayLose"]/(row["clayWin"] + row["clayLose"])
        subDf.loc[index,"grassLose"] = row["grassLose"]/(row["grassWin"] + row["grassLose"])
        
    return subDf


def addFieldWinLose(now, fwlr):
    
    now["P1_winningField"] = 0
    now["P1_losingField"] = 0
    now["P2_winningField"] = 0
    now["P2_losingField"] = 0
    
    for index, row in now.iterrows():
        value1 = 0
        value2 = 0
        if now.loc[index, "Surface_Hard"] == 1:
            value1=fwlr[fwlr["player"] == row["Player1"]]["hardWin"].tolist()[0]
            value2=fwlr[fwlr["player"] == row["Player2"]]["hardWin"].tolist()[0]
            
        elif now.loc[index, "Surface_Grass"] == 1:
            value=fwlr[fwlr["player"] == row["Player1"]]["grassWin"].tolist()[0]
            value2=fwlr[fwlr["player"] == row["Player2"]]["grassWin"].tolist()[0]

        elif now.loc[index, "Surface_Clay"] == 1:
            value=fwlr[fwlr["player"] == row["Player1"]]["clayWin"].tolist()[0]
            value2=fwlr[fwlr["player"] == row["Player2"]]["clayWin"].tolist()[0]
    
    
        now.loc[index,"P1_winningField"] = value1
        now.loc[index,"P1_losingField"] = 1 - value1
        
        now.loc[index,"P2_winningField"] = value2
        now.loc[index,"P2_losingField"] = 1 - value2
    
    
def addFieldWR (now):
    copy =  now.loc[:,["Winner", "Loser","Surface"]]
    fwlr = fieldWinLoseRateo(fieldWinLose(),copy)
    now = generalFormatting(now)
    addFieldWinLose(now, fwlr)


In [90]:
# fwlr.to_csv("fieldWinLoseRateo2018.csv", sep='\t', encoding='utf-8')

In [ ]:
fwlr[fwlr["player"] == "De Minaur A."]

In [108]:

now = generalFormatting(now)

In [129]:
from function import addFieldWR
now = addFieldWR(now)

1362


In [130]:
now

,Tournament,Date,Series,Round,Player1,Player2,P1_Rank,P2_Rank,P1_Pts,P2_Pts,...,Surface_Clay,Surface_Grass,Surface_Hard,Best of_3,Best of_5,target,P1_winningField,P1_losingField,P2_winningField,P2_losingField
0,12,2018-12-31,0,1,Dimitrov G.,Nishioka Y.,19.0,75.0,1835.0,701.0,...,0,0,1,1,0,0,0.993976,0.006024,0.954545,0.045455
1,12,2018-12-31,0,1,Raonic M.,Bedene A.,18.0,67.0,1855.0,780.0,...,0,0,1,1,0,0,0.994737,0.005263,0.971429,0.028571
2,12,2018-12-31,0,1,Kecmanovic M.,Mayer L.,131.0,56.0,433.0,895.0,...,0,0,1,1,0,0,0.500000,0.500000,0.974359,0.025641
3,12,2018-12-31,0,1,Millman J.,Sandgren T.,38.0,61.0,1083.0,814.0,...,0,0,1,1,0,0,0.960000,0.040000,0.750000,0.250000
4,12,2018-12-31,0,1,Uchiyama Y.,Humbert U.,185.0,102.0,275.0,572.0,...,0,0,1,1,0,0,0.666667,0.333333,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2605,35,2019-11-15,3,0,Nadal R.,Tsitsipas S.,1.0,6.0,9585.0,4000.0,...,0,0,1,1,0,0,0.500000,0.500000,0.500000,0.500000
2606,35,2019-11-15,3,0,Zverev A.,Medvedev D.,7.0,4.0,2945.0,5705.0,...,0,0,1,1,0,0,0.500000,0.500000,0.500000,0.500000
2607,35,2019-11-16,3,6,Tsitsipas S.,Federer R.,6.0,3.0,4000.0,6190.0,...,0,0,1,1,0,0,0.500000,0.500000,0.500000,0.500000
2608,35,2019-11-16,3,6,Thiem D.,Zverev A.,5.0,7.0,5025.0,2945.0,...,0,0,1,1,0,0,0.500000,0.500000,0.500000,0.500000
